In [65]:
import pandas as pd
import openpyxl as op
from tqdm.notebook import tqdm
from helpers.strings import keys_to_snake_case, keys_to_camel_case
from rest_framework_json_api.utils import format_field_names

In [66]:
company = Company.objects.get(name__exact="Arteris Via Paulista - Faixa de Domínio")
filename = company.name.replace(' ','_')+'_Formularios'
pd.DataFrame().to_excel(filename+'.xlsx')
wb=op.load_workbook(filename+'.xlsx')
wb['Sheet1'].title = "Disclaimer"
company,filename

(<Company: ec364a8e-b335-4729-8ac8-50ccabd73459: Arteris Via Paulista - Faixa de Domínio>,
 'Arteris_Via_Paulista_-_Faixa_de_Domínio_Formularios')

In [67]:
occs = OccurrenceType.objects.filter(company=company,occurrence_kind=2).exclude(occurrence_kind__isnull=True)
print(occs.count())

2


In [68]:
for match in tqdm(occs):
    for index, field in enumerate(match.form_fields['fields']):
        match.form_fields['fields'][index]=format_keys(keys_to_camel_case(field))
    for indexg, field_group in enumerate(match.form_fields['groups']):
        match.form_fields['groups'][indexg]=format_keys(keys_to_camel_case(field_group))
    match.save()

  0%|          | 0/2 [00:00<?, ?it/s]

In [69]:
def generate_select(field):
    result={}
    values=field["selectOptions"]["options"]
    
    for i,value in enumerate(values):
        result["options_"+str(i)]=value.get("name")

    return result

In [86]:
def generate_array(field):
    
    values=field["innerFields"]
    i=[{}]
    for value in values:
        result={"displayName":value['displayName'],"apiName":value['apiName']}
        if ((value["dataType"]=="select") and ("options" in value["selectOptions"])) or ((value["dataType"]=="selectMultiple") and ("options" in value["selectOptions"])):
            result["selectOptions"]=generate_select(value)
            print(result["selectOptions"])
        i.append(result)    
    return i

In [88]:
def generate_export(field):
    result={"displayName":field['displayName'],"apiName":field['apiName']}
    
    if ((field["dataType"]=="select") and ("options" in field["selectOptions"])) or ((field["dataType"]=="selectMultiple") and ("options" in field["selectOptions"])):
        result["selectOptions"]=generate_select(field)
    elif field["dataType"]=="arrayOfObjects":
        valores = generate_array(field)
        print(valores)
        result["temp"] = valores
#         result["arrayOfObjects_apiName"] = valores['apiName']
#         result["arrayOfObjects_displayName"] = valores['displayName']
#         result["arrayOfObjects_selectOptions"] = valores.get('selectOptions')
    
    return result

In [89]:
def format_keys(obj, format_type="camelize"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(value, dict):
                obj[key] = format_field_names(
                            format_keys(value), format_type
                            )
            elif isinstance(value, list):
                list_temp = value.copy()
                for i, item in enumerate(value):
                    if isinstance(item, dict):
                        list_temp[i] = format_field_names(
                        format_keys(item), format_type
                        )
                    else:
                        list_temp[i] = item
                obj[key] = list_temp
    return obj

In [90]:
sheetname =""
for dataOcc in tqdm(occs):
    index=[{}]
    sheetname = dataOcc.name.translate(str.maketrans('', '', './,!?'))[:15]
    for indexDict,fieldDict in enumerate(dataOcc.form_fields['fields']):
        index.append(generate_export(fieldDict))
        
    df2 = pd.DataFrame(data=index)
    
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df2.to_excel(writer,sheet_name=sheetname,na_rep="-")

  0%|          | 0/2 [00:00<?, ?it/s]

[{}, {'displayName': 'Anuidade', 'apiName': 'annuityStep'}, {'displayName': 'Inicio do período', 'apiName': 'annuityBeginStep'}, {'displayName': 'Fim do período', 'apiName': 'annuityEndStep'}, {'displayName': 'Valor da anuidade', 'apiName': 'annuityValueStep'}, {'displayName': 'Valor pago no ano', 'apiName': 'annuityValuePaidStep'}]
[{}, {'displayName': 'Numero da carta', 'apiName': 'letterNumberNotificationStep'}, {'displayName': 'Data', 'apiName': 'dataNotificationStep'}, {'displayName': 'Número do protocolo', 'apiName': 'arNotificationStep'}, {'displayName': 'Recebido por', 'apiName': 'recebidoPorNotificationStep'}]
[{}, {'displayName': 'Numero da carta', 'apiName': 'letterNumberNotificationStep'}, {'displayName': 'Data', 'apiName': 'dataNotificationStep'}, {'displayName': 'Número do protocolo', 'apiName': 'arNotificationStep'}, {'displayName': 'Recebido por', 'apiName': 'recebidoPorNotificationStep'}]
[{}, {'displayName': 'Numero da carta', 'apiName': 'letterNumberCienceStep'}, {'d

In [91]:
df2

,displayName,apiName,selectOptions,temp
0,NaN,NaN,NaN,NaN
1,Código Do Acesso,codAcesso,NaN,NaN
2,Código Novo,codNovo,NaN,NaN
3,Lote,lote,NaN,NaN
4,Tipo Da Obra,tipoDaObra,NaN,NaN
5,Localização,localization,NaN,NaN
6,Latitude,latitude,NaN,NaN
7,Longitude,longitude,NaN,NaN
8,Cidade,city,"{'options_0': 'Coronel Macedo', 'options_1': '...",NaN
9,Interessado,interessado,NaN,NaN
